In [ ]:
! load_ext autoreload
! autoreload 2
from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler
import torch
import os
import math
import re
from itertools import groupby
from typing import Callable, Dict, List, Optional, Set, Tuple, Type, Union

import numpy as np
import PIL
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
sys.path.append('../')
from lora_diffusion import LoraInjectedConv2d, LoraInjectedLinear, patch_pipe, tune_lora_scale, parse_safeloras
from lora_diffusion.lora import _find_modules, UNET_CROSSATTN_TARGET_REPLACE, DEFAULT_TARGET_REPLACE
from reg_lora.visual import visualize_images
import safetensors

# os.environ["DISABLE_TELEMETRY"] = 'YES'
model_id = "runwayml/stable-diffusion-v1-5"
device = torch.device("cuda:3")
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16,local_ckpt_files_only=True,revision='39593d5650112b4cc580433f6b0435385882d819').to(device)
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
torch.manual_seed(0)


### 在attn_layer中插入hook函数, 可视化attn_map 

In [ ]:
# %%capture
import os
import sys
sys.path.append('/home/zhicai/poseVideo/lora-master/')
path = 'lora_output/checkpoints/debug_decay_0.001/lora_weight_e31_s5000.safetensors'
path = 'lora_output/checkpoints/output_dog_Ti-clip_Nonorm_3e-5/lora_weight_e62_s10000.safetensors'
prompt = "'a <krk1> dog in grand canyon'"

command = f"python ../reg_lora/vis_attn_map.py --lora_path {path} --prompt {prompt} --gpu 0 --name up_blocks.2.attentions.1.transformer_blocks.0.attn2"
print(command)
os.system(command)

### 计算TI后cpation-image的Clip score

In [ ]:
import re
import os
import copy
import torch

from PIL import Image
from transformers import CLIPProcessor, CLIPModel

lora_dir = '../lora_output/checkpoints/debug_decay_0.001'
root_img_path = '/home/zhicai/poseVideo/Text-regularized-customization/custom_data/data/dog'
ckpt_pattern = r'lora_weight_e\d+_s\d{3,4}.safetensors'
captions = ["photo of a <krk1> dog", "photo of a dog"]
device = 'cuda:2'
weight_dtype = torch.float32


model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device,dtype = weight_dtype)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

images = [Image.open(os.path.join(root_img_path, img_path)) for img_path in os.listdir(root_img_path)]
    
_ckpt_files = os.listdir(lora_dir)
ckpt_files = [f for f in _ckpt_files if re.match(ckpt_pattern, f)]

probs_list = []
for _file in sorted(ckpt_files,
                    key = lambda x: int(re.match(r'.*e([0-9]+).*', x.split('/')[-1])[1]), 
                    reverse=False):
    
    lora_path = os.path.join(lora_dir, _file)
    print(f"loading from: {lora_path}")
    patch_pipe( 
        pipe,
        lora_path,
        patch_text=False,
        patch_ti=True,
        patch_unet=True,
        filter_crossattn_str = 'cross+self'
    )

    processor.tokenizer = pipe.tokenizer
    model.text_model = copy.deepcopy(pipe.text_encoder.text_model.to(device, dtype=weight_dtype))
    # token_embedding = model.text_model.embeddings.token_embedding
    # token_embedding.weight.data[49408] = token_embedding.weight.data[42170]    
    inputs = processor(text=captions, images=images, return_tensors="pt", padding=True).to(device)
    outputs = model(**inputs)
    
    image_embeds = outputs.image_embeds
    input_ids = inputs.input_ids
    last_hidden_state = outputs.text_model_output.last_hidden_state
    text_embeds = last_hidden_state[torch.arange(last_hidden_state.size(0), device=input_ids.device),
                                    [(row == 49407).nonzero().min() for row in input_ids]]
    text_embeds = text_embeds / text_embeds.norm(p=2, dim=-1, keepdim=True)
    logit_scale = model.logit_scale.exp()
    logits_per_text = torch.matmul(text_embeds, image_embeds.t()) * logit_scale
    logits_per_image = logits_per_text.t()
    
    probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities
    probs_list.append(probs[:,0].mean().item())

In [ ]:
import matplotlib.pyplot as plt 
plt.plot(np.arange(len(probs_list)) * 200, probs_list)
plt.show()

In [ ]:
import matplotlib.pyplot as plt 
plt.plot(np.arange(len(probs_list)) * 200, probs_list)
plt.show()

### 统计Lora中的self和cross attn模块中的Linear-layer权重漂移和对文本映射后的语义漂移

In [ ]:
target_dir = '../output_dog'
test_prompts = ['a dog','a cat','a teddybear','a chair']
reg_prompt = ['photo of a dog']

lora_weight_norm_list_cross = []
lora_weight_norm_list_self = []
lora_context_reg_resides_list = []

# 找到cross_attn 和 self_attn的位置
cnt = 0
cross_idxs = []
self_idxs = []
for target, name, module in _find_modules(pipe.unet, DEFAULT_TARGET_REPLACE):
    if module.in_features == 768:
        cross_idxs.append(cnt)
    if module.in_features != 320 and (name == 'to_k' or name == 'to_v'):
        self_idxs.append(cnt)
    cnt += 1 

c_reg = pipe._encode_prompt(reg_prompt,device,1,False)
group = [ckpt_file  for ckpt_file in os.listdir(target_dir) if '_s' in ckpt_file and 'lora_weight' in ckpt_file]
print(group)
sorted_group = sorted(group, key = lambda x: int(re.findall(r'.*s(\d+).*',x)[0]))

for ckpt_file in sorted_group:
        cur_step_list_cross = []
        cur_step_context_reg_reside = []
        cur_step_list_self = []
        ckpt = safetensors.safe_open(os.path.join(target_dir,ckpt_file), framework="pt", device="cpu")
        lora = parse_safeloras(ckpt)  
        for cross_idx , self_idx in zip(cross_idxs,self_idxs):
                up_weight_self = lora['unet'][0][2*self_idx].to(device)
                down_weight_self = lora['unet'][0][2*self_idx+1].to(device)
                up_weight = lora['unet'][0][2*cross_idx].to(device)
                down_weight = lora['unet'][0][2*cross_idx+1].to(device)
                cur_step_lora_project_self = down_weight_self.transpose(0,1) @ up_weight_self.transpose(0,1)
                cur_step_lora_project_cross = down_weight.transpose(0,1) @ up_weight.transpose(0,1)
                cur_step_lora_reside = c_reg @ cur_step_lora_project_cross
                cur_step_context_reg_reside.append(torch.norm(cur_step_lora_reside).cpu().item())
                cur_step_list_cross.append(torch.norm(cur_step_lora_project_cross).cpu().item()) 
                cur_step_list_self.append(torch.norm(cur_step_lora_project_self).cpu().item())
        lora_weight_norm_list_cross.append(np.mean(cur_step_list_cross))
        lora_context_reg_resides_list.append(np.mean(cur_step_context_reg_reside))
        lora_weight_norm_list_self.append(np.mean(cur_step_list_self))

print(lora_weight_norm_list_cross[4:],'\n',lora_weight_norm_list_self[4:],'\n',lora_context_reg_resides_list[4:])

## Visualization figures

In [ ]:
import copy
lora_ckpts = ['../lora_output/checkpoints/output_dog_Ti-clip_norm/lora_weight_e31_s5000.safetensors']
prompts = ['photo of a <krk1> dog and a cat', 'photo of a cat and a <krk1> dog']
bs = 4
weight_dtype = torch.float16
device = "cuda:2"
pipe_copy = copy.deepcopy(pipe).to(device)
torch.manual_seed(0)
patch_pipe(
    pipe_copy,
    lora_ckpts[0],
    patch_text=False,
    patch_ti=True,
    patch_unet=True,
    filter_crossattn_str = 'cross+self'
)
# pipe.unet
tune_lora_scale(pipe_copy.unet, 0)
pipe_copy.text_encoder.text_model.to(device, dtype=weight_dtype)
# tune_lora_scale(pipe_copy.text_encoder, 1)
for prompt in prompts:
    prompt = [prompt]*bs
    img = pipe_copy(prompt = prompt, num_inference_steps=50, guidance_scale=6).images
    visualize_images(img,outpath='figure/', nrow=4, show=True,save=False)

In [ ]:
import copy
lora_ckpts = ['../lora_output/checkpoints/debug_decay_mask_0.001/lora_weight_e31_s5000.safetensors']
prompts = ['photo of a <krk1> dog and a cat', 'photo of a cat and a <krk1> dog']
bs = 4
weight_dtype = torch.float16
device = "cuda:2"
pipe_copy = copy.deepcopy(pipe).to(device)
torch.manual_seed(0)
patch_pipe(
    pipe_copy,
    lora_ckpts[0],
    patch_text=False,
    patch_ti=True,
    patch_unet=True,
    filter_crossattn_str = 'cross+self'
)
# pipe.unet
tune_lora_scale(pipe_copy.unet, 0)
pipe_copy.text_encoder.text_model.to(device, dtype=weight_dtype)
# tune_lora_scale(pipe_copy.text_encoder, 1)
for prompt in prompts:
    prompt = [prompt]*bs
    img = pipe_copy(prompt = prompt, num_inference_steps=50, guidance_scale=6).images
    visualize_images(img,outpath='figure/', nrow=4, show=True,save=False)

In [ ]:
import copy
lora_ckpts = ['../lora_output/checkpoints/NT-clipDecay-mask']
prompts = ['photo of a <krk1> dog', 'photo of a <krk1> dog swimming in a pool', 'photo of a <krk1> dog in grand canyon']
bs = 4
weight_dtype = torch.float16
device = "cuda:2"
pipe_copy = copy.deepcopy(pipe).to(device)
torch.manual_seed(0)
patch_pipe(
    pipe_copy,
    lora_ckpts[0],
    patch_text=False,
    patch_ti=True,
    patch_unet=True,
    filter_crossattn_str = 'cross+self'
)
# pipe.unet
tune_lora_scale(pipe_copy.unet, 1)
pipe_copy.text_encoder.text_model.to(device, dtype=weight_dtype)
# tune_lora_scale(pipe_copy.text_encoder, 1)
for prompt in prompts:
    prompt = [prompt]*bs
    img = pipe_copy(prompt = prompt, num_inference_steps=50, guidance_scale=6).images
    visualize_images(img,outpath='figure/', nrow=4, show=True,save=False)

In [ ]:
import copy
lora_ckpts = ['../lora_output/checkpoints/NT-Decay/lora_weight_e12_s2000.safetensors']
prompts = ['photo of a <krk1> dog', 'photo of a <krk1> dog swimming in a pool']
bs = 4
weight_dtype = torch.float16
device = "cuda:2"
pipe_copy = copy.deepcopy(pipe).to(device)
torch.manual_seed(0)
patch_pipe(
    pipe_copy,
    lora_ckpts[0],
    patch_text=False,
    patch_ti=True,
    patch_unet=True,
    filter_crossattn_str = 'cross+self'
)
# pipe.unet
tune_lora_scale(pipe_copy.unet, 1)
pipe_copy.text_encoder.text_model.to(device, dtype=weight_dtype)
# tune_lora_scale(pipe_copy.text_encoder, 1)
for prompt in prompts:
    prompt = [prompt]*bs
    img = pipe_copy(prompt = prompt, num_inference_steps=50, guidance_scale=6).images
    visualize_images(img,outpath='figure/', nrow=4, show=True,save=False)

## Masked attention projection

In [ ]:
from diffusers.pipelines.stable_diffusion import StableDiffusionPipelineOutput

# print(input_ids)
def _expand_mask(mask: torch.Tensor, dtype: torch.dtype, tgt_len: Optional[int] = None):
    """
    Expands attention_mask from `[bsz, seq_len]` to `[bsz, 1, tgt_seq_len, src_seq_len]`.
    """
    bsz, src_len = mask.size()
    tgt_len = tgt_len if tgt_len is not None else src_len

    expanded_mask = mask[:, None, None, :].expand(bsz, 1, tgt_len, src_len).to(dtype)

    inverted_mask = 1.0 - expanded_mask
    return inverted_mask.masked_fill(inverted_mask.to(torch.bool), torch.finfo(dtype).min)

def get_identifier_masked_causal_attention_mask(bs, seq_len, identifier_indice, class_token_len, dtype = torch.float16):
    mask = torch.empty(bs, seq_len, seq_len, dtype=dtype)
    mask.fill_(torch.tensor(torch.finfo(dtype).min))
    mask.triu_(1)  # zero out the lower diagonal
    mask = mask.unsqueeze(1)  # expand mask
    mask[:,:,identifier_indice, 1:max(identifier_indice,1)] = torch.finfo(dtype).min
    mask[:,:,identifier_indice+class_token_len+1:,identifier_indice] = torch.finfo(dtype).min
    return mask

def build_causal_attention_mask( bs, seq_len, dtype):
    mask = torch.empty(bs, seq_len, seq_len, dtype=dtype)
    mask.fill_(torch.tensor(torch.finfo(dtype).min))
    mask.triu_(1)  # zero out the lower diagonal
    mask = mask.unsqueeze(1)  # expand mask
    return mask

@torch.no_grad()
def generate_with_hidden_input(pipe, hidden_states ,num_images_per_prompt=1, num_inference_steps=50, guidance_scale=6):

    text_embeddings = hidden_states
    batch_size = hidden_states.size(0)
    do_classifier_free_guidance = guidance_scale > 1.0
    if do_classifier_free_guidance:
        uncond_tokens = [""] * batch_size
        uncond_input = pipe.tokenizer(
                uncond_tokens,
                padding="max_length",
                max_length=pipe.tokenizer.model_max_length,
                truncation=True,
                return_tensors="pt",
            )
        uncond_embeddings = pipe.text_encoder(uncond_input.input_ids.to(device), attention_mask = None)[0]
        bs_embed, seq_len, _ = text_embeddings.shape
        text_embeddings = text_embeddings.repeat(1, num_images_per_prompt, 1)
        text_embeddings = text_embeddings.view(bs_embed * num_images_per_prompt, seq_len, -1)
        uncond_embeddings = uncond_embeddings.repeat(1, num_images_per_prompt, 1)
        uncond_embeddings = uncond_embeddings.view(batch_size * num_images_per_prompt, seq_len, -1)
        text_embeddings = torch.cat([uncond_embeddings, text_embeddings])

    height =  pipe.unet.config.sample_size * pipe.vae_scale_factor
    width = pipe.unet.config.sample_size * pipe.vae_scale_factor
    pipe.scheduler.set_timesteps(num_inference_steps, device=device)
    timesteps = pipe.scheduler.timesteps

    # 5. Prepare latent variables
    num_channels_latents = pipe.unet.in_channels
    latents = pipe.prepare_latents(
        batch_size * num_images_per_prompt,
        num_channels_latents,
        height,
        width,
        text_embeddings.dtype,
        device,
        None,
        None,
    )

    # 6. Prepare extra step kwargs. TODO: Logic should ideally just be moved out of the pipeline
    extra_step_kwargs = pipe.prepare_extra_step_kwargs(None, 0.0)

    # 7. Denoising loop
    num_warmup_steps = len(timesteps) - num_inference_steps * pipe.scheduler.order
    with pipe.progress_bar(total=num_inference_steps) as progress_bar:
        for i, t in enumerate(timesteps):
            # expand the latents if we are doing classifier free guidance
            latent_model_input = torch.cat([latents] * 2) if do_classifier_free_guidance else latents
            latent_model_input = pipe.scheduler.scale_model_input(latent_model_input, t)

            # predict the noise residual
            noise_pred = pipe.unet(latent_model_input, t, encoder_hidden_states=text_embeddings).sample

            # perform guidance
            if do_classifier_free_guidance:
                noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
                noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

            # compute the previous noisy sample x_t -> x_t-1
            latents = pipe.scheduler.step(noise_pred, t, latents, **extra_step_kwargs).prev_sample

            # call the callback, if provided
            if i == len(timesteps) - 1 or ((i + 1) > num_warmup_steps and (i + 1) % pipe.scheduler.order == 0):
                progress_bar.update()

    # 8. Post-processing
    image = pipe.decode_latents(latents)

    # 9. Run safety checker
    # image, has_nsfw_concept = pipe.run_safety_checker(image, device, text_embeddings.dtype)
    # 10. Convert to PIL
    image = pipe.numpy_to_pil(image)

    return StableDiffusionPipelineOutput(images=image, nsfw_content_detected=None)


inputs = pipe.tokenizer(["photo of a red dog in grand canyon"], return_tensors="pt", max_length=77, truncation=True, padding='max_length')
input_ids = inputs.input_ids.to(device)
identifier_masked_causal_attention_mask = get_identifier_masked_causal_attention_mask(1, 77 ,4, 1, dtype= torch.float16).to(device)
causal_attention_mask = build_causal_attention_mask(1, 77, dtype= torch.float16).to(device)
clip_text_model = pipe.text_encoder.text_model
hidden_states = clip_text_model.embeddings(input_ids=input_ids)
outputs = clip_text_model.encoder(hidden_states,
                    causal_attention_mask = identifier_masked_causal_attention_mask,
                    # attention_mask = _expand_mask(attention_mask, torch.float16, 77).to(device),
                    output_attentions=True)
text_embedings = clip_text_model.final_layer_norm(outputs[0])
print(text_embedings.size())
imgs = generate_with_hidden_input(pipe, text_embedings, num_images_per_prompt = 4 , num_inference_steps=50, guidance_scale=6).images


In [ ]:
visualize_images(imgs,save=False,nrow=4,show=True)

In [1]:
import torch
device = 'cuda:2'
a = torch.randn(1,2,3)
b = torch.randn(1,2,3).to('cuda:0')
for _inputs in [a, b]:
    if isinstance(_inputs, torch.LongTensor):
        _inputs = _inputs.to(device)
    elif isinstance(_inputs, torch.FloatTensor):
        _inputs = _inputs.to(device)


/data/zhicai/miniconda3/envs/ldm/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cpu')